In [ ]:
import sys
sys.path.append("..")

import random
import math
import itertools
from copy import deepcopy
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union, Dict

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
import plotly.graph_objects as go
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.algo import GreedyLibrary
from src.util.image import *
from src.util import to_torch_device
from src.patchdb import PatchDB
from src.models.encoder import *

In [ ]:
!ls -l ../db/*.patchdb

# load patchdb

In [ ]:
db = PatchDB(
    #"../db/photos-1x32x32-2.patchdb", 
    #"../db/photos-bigpatch-1x32x32.patchdb",
    #"../db/kali-1x32x32.patchdb",
    #"../db/diverse-1x32x32-3b.patchdb",
    "../db/hyperplane-1x32x32-convall1.patchdb",
    
    #encoder=encoder,
    patch_shape=(1, 32, 32),
    verbose=True, limit=1_000_000, 
)
index = db.index()
print(f"{index.size} patches, {len(index.filenames())} images")

# select new encoder

In [ ]:
!ls -l ../models/encoder2d/

In [ ]:
if 0:
    # copy from training
    torch.save(
        torch.load("../checkpoints/rbm3/best.pt")["state_dict"],
        "../models/encoder2d/boltzman-1x32x32-128-photo.pt",
    )
    
#encoder = BoltzmanEncoder2d.from_torch("../models/encoder2d/boltzman-1x32x32-128-photo-300M.pt")
encoder = EncoderConv2d.from_torch("../models/encoder2d/conv-1x32x32-128-all1.pt")

encoder.device

In [ ]:
db2 = db.to_new_patchdb(
    filename="../db/hyperplane-inv-1x32x32-convall1.patchdb",
    encoder=encoder,
    transforms=[lambda x: 1. - x],
)